In [ ]:
%load_ext autoreload
%autoreload 1
# custom functions being developed interactively
%aimport assignment_utils_practice
import assignment_utils_practice as utils

import numpy as np
%matplotlib widget

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import mean_squared_error

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.activations import relu, linear
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)

from public_tests_a1 import *

tf.keras.backend.set_floatx('float64')
# from assigment_utils import *

tf.autograph.set_verbosity(0)

In [ ]:
# Generate some data
X, y, x_ideal, y_ideal = utils.gen_data(18, 2, 0.7)
print("X.shape", X.shape, "y.shape", y.shape)

# split the data using sklearn routine
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=1
)
print("X_train.shape", X_train.shape, "y_train.shape", y_train.shape)
print("X_test.shape", X_test.shape, "y_test.shape", y_test.shape)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(4, 4))
ax.plot(x_ideal, y_ideal, "--", color="orangered", label="y_ideal", lw=1)
ax.set_title("Training, Test", fontsize=14)
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.scatter(X_train, y_train, marker="o", color="red", label="train")
ax.scatter(X_test, y_test, marker="o", color="blue", label="test")
ax.legend(loc="upper left")
utils.format_plot_interface(fig.canvas)
plt.show()

In [ ]:
# UNQ_C1
# GRADED CELL: eval_mse
def eval_mse(y, yhat):
    """
    Calculate the mean squared error on a data set.
    Args:
      y    : (ndarray  Shape (m,) or (m,1))  target value of each example
      yhat : (ndarray  Shape (m,) or (m,1))  predicted value of each example
    Returns:
      err: (scalar)
    """
    m = len(y)
    err = 0.0
    for i in range(m):
        ### START CODE HERE ###
        sq_error = (y - yhat) ** 2
        err = np.sum(sq_error) / (2 * m)
    ### END CODE HERE ###

    return err

In [ ]:
y_hat = np.array([2.4, 4.2])
y_tmp = np.array([2.3, 4.1])
eval_mse(y_hat, y_tmp)

# BEGIN UNIT TEST
test_eval_mse(eval_mse)
# END UNIT TEST

In [ ]:
degree = 10
lmodel = utils.lin_model(degree)
lmodel.fit(X_train, y_train)

# predict on training data, find training error
yhat = lmodel.predict(X_train)
err_train = lmodel.mse(y_train, yhat)

# predict on test data, find error
yhat = lmodel.predict(X_test)
err_test = lmodel.mse(y_test, yhat)

In [ ]:
print(f"Training Error = {err_train:0.2f}, Test Error = {err_test:0.2f}")

In [ ]:
# plot predictions over data range
x = np.linspace(0, int(X.max()), 100)
y_pred = lmodel.predict(x).reshape(-1, 1)

utils.plt_train_test(
    X_train, y_train, X_test, y_test, x, y_pred, x_ideal, y_ideal, degree, buff=0.05
)

In [ ]:
# Generate  data
X, y, x_ideal, y_ideal = utils.gen_data(m=40, seed=5, scale=0.7)
print("X.shape", X.shape, "y.shape", y.shape)

# Split the data using sklearn routine
X_train, X_, y_train, y_ = train_test_split(X, y, test_size=0.40, random_state=1)
X_cv, X_test, y_cv, y_test = train_test_split(X_, y_, test_size=0.50, random_state=1)
print(f"X_train.shape = {X_train.shape}, y_train.shape = {y_train.shape}")
print(f"X_cv.shape = {X_cv.shape}, y_train.shape = {y_cv.shape}")
print(f"X_test.shape = {X_test.shape}, y_train.shape = {y_test.shape}")

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(4, 4))
utils.format_plot_interface(fig.canvas)
ax.plot(x_ideal, y_ideal, "--", color="orangered", lw=1, label="y_ideal")
ax.set_title("Training, CV, Test", fontsize=14)
ax.set_xlabel("x")
ax.set_ylabel("y")

ax.scatter(X_train, y_train, color="red", label="train")
ax.scatter(X_cv, y_cv, color=utils.dlc["dlorange"], label="cv")
ax.scatter(X_test, y_test, color=utils.dlc["dlblue"], label="test")
ax.legend(loc="upper left")
plt.show()

In [ ]:
max_degree = 9
err_train = np.zeros(max_degree)
err_cv = np.zeros(max_degree)
x = np.linspace(0, int(X.max()), 100)
y_pred = np.zeros((100, max_degree))

for degree in range(max_degree):
    lmodel = utils.lin_model(degree + 1)
    lmodel.fit(X_train, y_train)
    yhat = lmodel.predict(X_train)
    err_train[degree] = lmodel.mse(y_train, yhat)
    yhat = lmodel.predict(X_cv)
    err_cv[degree] = lmodel.mse(y_cv, yhat)
    y_pred[:, degree] = lmodel.predict(x)

optimal_degree = np.argmin(err_cv) + 1

In [ ]:
plt.close("all")
utils.plt_optimal_degree(
    X_train,
    y_train,
    X_cv,
    y_cv,
    x,
    y_pred,
    x_ideal,
    y_ideal,
    err_train,
    err_cv,
    optimal_degree,
    max_degree,
)

In [ ]:
lambda_range = np.array([0.0, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100])
num_steps = len(lambda_range)
degree = 10
err_train = np.zeros(num_steps)
err_cv = np.zeros(num_steps)
x = np.linspace(0, int(X.max()), 100)
y_pred = np.zeros((100, num_steps))  # columns are lines to plot

for i in range(num_steps):
    lambda_ = lambda_range[i]
    lmodel = utils.lin_model(degree, regularization=True, lambda_=lambda_)
    lmodel.fit(X_train, y_train)
    yhat = lmodel.predict(X_train)
    err_train[i] = lmodel.mse(y_train, yhat)
    yhat = lmodel.predict(X_cv)
    err_cv[i] = lmodel.mse(y_cv, yhat)
    y_pred[:, i] = lmodel.predict(x)

optimal_reg_idx = np.argmin(err_cv)

In [ ]:
plt.close("all")
utils.plt_tune_regularization(
    X_train,
    y_train,
    X_cv,
    y_cv,
    x,
    y_pred,
    err_train,
    err_cv,
    optimal_reg_idx,
    lambda_range,
)

In [ ]:
plt.close("all")
X_train, y_train, X_cv, y_cv, x, y_pred, err_train, err_cv, m_range, degree = (
    utils.tune_m()
)
utils.plt_tune_m(
    X_train, y_train, X_cv, y_cv, x, y_pred, err_train, err_cv, m_range, degree
)

In [ ]:
# Generate and split data set
X, y, centers, classes, std = utils.gen_blobs()

# split the data. Large CV population for demonstration
X_train, X_, y_train, y_ = train_test_split(X, y, test_size=0.50, random_state=1)
X_cv, X_test, y_cv, y_test = train_test_split(X_, y_, test_size=0.20, random_state=1)
print(
    "X_train.shape:",
    X_train.shape,
    "X_cv.shape:",
    X_cv.shape,
    "X_test.shape:",
    X_test.shape,
)

In [ ]:
plt.close("all")
utils.plt_train_eq_dist(X_train, y_train, classes, X_cv, y_cv, centers, std)

In [ ]:
# UNQ_C2
# GRADED CELL: eval_cat_err
def eval_cat_err(y, yhat):
    """
    Calculate the categorization error
    Args:
      y    : (ndarray  Shape (m,) or (m,1))  target value of each example
      yhat : (ndarray  Shape (m,) or (m,1))  predicted value of each example
    Returns:|
      cerr: (scalar)
    """
    # m = len(y)
    # incorrect = 0
    # for i in range(m):
    # ### START CODE HERE ###
    #     err = 1 if y[i] != yhat[i] else 0
    #     incorrect = incorrect + err
    #     cerr = incorrect/m
    # ### END CODE HERE ###
    # return(cerr)
    err = np.where(y == yhat, 0, 1)
    return np.mean(err)

In [ ]:
y_hat = np.array([1, 2, 0])
y_tmp = np.array([1, 2, 3])
print(
    f"categorization error {np.squeeze(eval_cat_err(y_hat, y_tmp)):0.3f}, expected:0.333"
)
y_hat = np.array([[1], [2], [0], [3]])
y_tmp = np.array([[1], [2], [1], [3]])
print(
    f"categorization error {np.squeeze(eval_cat_err(y_hat, y_tmp)):0.3f}, expected:0.250"
)

# BEGIN UNIT TEST
test_eval_cat_err(eval_cat_err)
# END UNIT TEST

In [ ]:
# UNQ_C3
# GRADED CELL: model
import logging

logging.getLogger("tensorflow").setLevel(logging.ERROR)

tf.random.set_seed(1234)
model = Sequential(
    [
        ### START CODE HERE ###
        Dense(units=120, activation="relu"),
        Dense(units=40, activation="relu"),
        Dense(units=6, activation="linear"),
        ### END CODE HERE ###
    ],
    name="Complex",
)
model.compile(
    ### START CODE HERE ###
    loss=SparseCategoricalCrossentropy(from_logits=True),
    optimizer=Adam(learning_rate=0.01),
    ### END CODE HERE ###
)

In [ ]:
# BEGIN UNIT TEST
model.fit(X_train, y_train, epochs=1000)
# END UNIT TEST

In [ ]:
# BEGIN UNIT TEST
model.summary()

# model_test(model, classes, X_train.shape[1])
# END UNIT TEST

In [ ]:
tf.keras.utils.disable_interactive_logging()
# make a model for plotting routines to call
model_predict_fn = lambda Xl: np.argmax(
    tf.nn.softmax(model.predict(Xl)).numpy(), axis=1
)
utils.plt_nn(
    model_predict_fn, X_train, y_train, classes, X_cv, y_cv, suptitle="Complex Model"
)

In [ ]:
training_cerr_complex = eval_cat_err(y_train, model_predict_fn(X_train))
cv_cerr_complex = eval_cat_err(y_cv, model_predict_fn(X_cv))
print(f"categorization error, training, complex model: {training_cerr_complex:0.3f}")
print(f"categorization error, cv,       complex model: {cv_cerr_complex:0.3f}")

In [ ]:
# UNQ_C4
# GRADED CELL: model_s

tf.random.set_seed(1234)
model_s = Sequential(
    [
        ### START CODE HERE ###
        Dense(units=6, activation="relu"),
        Dense(units=6, activation="linear"),
        ### END CODE HERE ###
    ],
    name="Simple",
)
model_s.compile(
    ### START CODE HERE ###
    loss=SparseCategoricalCrossentropy(from_logits=True),
    optimizer=Adam(learning_rate=0.01),
    ### START CODE HERE ###
)

In [ ]:
tf.keras.utils.enable_interactive_logging()

# BEGIN UNIT TEST
model_s.fit(X_train, y_train, epochs=1000)
# END UNIT TEST

In [ ]:
# BEGIN UNIT TEST
model_s.summary()

# model_s_test(model_s, classes, X_train.shape[1])
# END UNIT TEST

In [ ]:
tf.keras.utils.disable_interactive_logging()
# make a model for plotting routines to call
model_predict_s = lambda Xl: np.argmax(
    tf.nn.softmax(model_s.predict(Xl)).numpy(), axis=1
)
utils.plt_nn(
    model_predict_s, X_train, y_train, classes, X_cv, y_cv, suptitle="Simple Model"
)

In [ ]:
training_cerr_simple = eval_cat_err(y_train, model_predict_s(X_train))
cv_cerr_simple = eval_cat_err(y_cv, model_predict_s(X_cv))
print(
    f"categorization error, training, simple model, {training_cerr_simple:0.3f}, complex model: {training_cerr_complex:0.3f}"
)
print(
    f"categorization error, cv,       simple model, {cv_cerr_simple:0.3f}, complex model: {cv_cerr_complex:0.3f}"
)

In [ ]:
# UNQ_C5
# GRADED CELL: model_r

tf.random.set_seed(1234)
model_r = Sequential(
    [
        ### START CODE HERE ###
        Dense(
            units=120,
            activation="relu",
            kernel_regularizer=tf.keras.regularizers.l2(0.1),
        ),
        Dense(
            units=40,
            activation="relu",
            kernel_regularizer=tf.keras.regularizers.l2(0.1),
        ),
        Dense(units=6, activation="linear"),
        ### START CODE HERE ###
    ],
    name=None,
)
model_r.compile(
    ### START CODE HERE ###
    loss=SparseCategoricalCrossentropy(from_logits=True),
    optimizer=Adam(learning_rate=0.01),
    ### START CODE HERE ###
)

In [ ]:
tf.keras.utils.enable_interactive_logging()
# BEGIN UNIT TEST
model_r.fit(X_train, y_train, epochs=1000)
# END UNIT TEST

In [ ]:
# BEGIN UNIT TEST
model_r.summary()

# model_r_test(model_r, classes, X_train.shape[1])
# END UNIT TEST

In [ ]:
# make a model for plotting routines to call
model_predict_r = lambda Xl: np.argmax(
    tf.nn.softmax(model_r.predict(Xl)).numpy(), axis=1
)

utils.plt_nn(
    model_predict_r, X_train, y_train, classes, X_cv, y_cv, suptitle="Regularized"
)

In [ ]:
training_cerr_reg = eval_cat_err(y_train, model_predict_r(X_train))
cv_cerr_reg = eval_cat_err(y_cv, model_predict_r(X_cv))
test_cerr_reg = eval_cat_err(y_test, model_predict_r(X_test))
print(
    f"categorization error, training, regularized: {training_cerr_reg:0.3f}, simple model, {training_cerr_simple:0.3f}, complex model: {training_cerr_complex:0.3f}"
)
print(
    f"categorization error, cv,       regularized: {cv_cerr_reg:0.3f}, simple model, {cv_cerr_simple:0.3f}, complex model: {cv_cerr_complex:0.3f}"
)

In [ ]:
tf.random.set_seed(1234)
lambdas = [0.0, 0.001, 0.01, 0.05, 0.1, 0.2, 0.3]
models = [None] * len(lambdas)

for i in range(len(lambdas)):
    lambda_ = lambdas[i]
    models[i] = Sequential(
        [
            Dense(
                120,
                activation="relu",
                kernel_regularizer=tf.keras.regularizers.l2(lambda_),
            ),
            Dense(
                40,
                activation="relu",
                kernel_regularizer=tf.keras.regularizers.l2(lambda_),
            ),
            Dense(classes, activation="linear"),
        ]
    )
    models[i].compile(
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        optimizer=tf.keras.optimizers.Adam(0.01),
    )

    models[i].fit(X_train, y_train, epochs=1000)
    print(f"Finished lambda = {lambda_}")

In [ ]:
plt.close("all")
utils.plot_iterate(lambdas, models, X_train, y_train, X_cv, y_cv)

In [ ]:
plt.close("all")
utils.plt_compare(X_test, y_test, classes, model_predict_s, model_predict_r, centers)